In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import KFold
from sklearn.metrics import log_loss
from sklearn.preprocessing import  OneHotEncoder
from sklearn.preprocessing import StandardScaler
from scipy import stats

In [2]:
def imputeDFColsUsingMedian(dataFrame,cols):
    for col in cols:
        medianOfCol=np.nanmedian(dataFrame[col])
        dataFrame[col].fillna(medianOfCol,inplace=True)
def imputeDFColsUsingMean(dataFrame,cols):
    for col in cols:
        meanOfCol=np.nanmean(dataFrame[col])
        dataFrame[col].fillna(meanOfCol,inplace=True)
def scaleFeature(dataFrame,col):
    maxVal=np.max(dataFrame[col])
    minVal=np.min(dataFrame[col])
    scaledDenom=maxVal-minVal
    dataFrame[col]=(dataFrame[col]-minVal)/scaledDenom
def labelEncodeFeats(dataFrame,listOfFeats):
    for feat in listOfFeats:
        labelEncoder=LabelEncoder()
        encodedFeatValues=labelEncoder.fit_transform(dataFrame[feat])
        dataFrame[feat]=encodedFeatValues
def OneHotEncodeFeats(dataFrame,listOfFeats,ctgrcl_ftrs_msk):
    labelEncodeFeats(dataFrame,listOfFeats)
    oneHotEncoder=OneHotEncoder(categorical_features=ctgrcl_ftrs_msk,sparse=False)
    oneHotEncodedFeats=oneHotEncoder.fit_transform(dataFrame)
    return oneHotEncodedFeats

In [ ]:
app_events = pd.read_csv('../Data/app_events.csv')
app_labels = pd.read_csv('../Data/app_labels.csv')
events = pd.read_csv('../Data/events.csv')
events.timestamp=events.timestamp.map(lambda x:pd.Timestamp(x).value)
eventsGrpdByDeviceId=events.groupby('device_id')
gender_age_train = pd.read_csv('../Data/gender_age_train.csv')
gender_age_test = pd.read_csv('../Data/gender_age_test.csv')
label_categories = pd.read_csv('../Data/label_categories.csv')
phone_brand_device_model = pd.read_csv('../Data/phone_brand_device_model.csv',encoding='utf-8')
phone_brand_device_model = phone_brand_device_model.drop_duplicates('device_id',keep='first')

In [4]:
def joinBrandDeviceModel(deviceIDFrame,brandDeviceModelFrame):
    mergedDF=deviceIDFrame.merge(brandDeviceModelFrame[['device_id','phone_brand','device_model']], 
                                 how='left',on='device_id')
    mergedDF['phone_brand'].fillna('',inplace=True)
    mergedDF['device_model'].fillna('',inplace=True)
    return mergedDF
device_brand_model_trainDF=joinBrandDeviceModel(gender_age_train,phone_brand_device_model)
device_brand_model_testDF=joinBrandDeviceModel(gender_age_test,phone_brand_device_model)

In [5]:
def avgEventDuration(listOfTimeStamps):
    if(len(listOfTimeStamps)<1):
        return 0
    return (np.max(listOfTimeStamps)-np.min(listOfTimeStamps))/len(listOfTimeStamps)
def stdEventDuration(listOfTimeStamps):
    if(len(set(listOfTimeStamps))<=1):
        return 0
    return np.std(listOfTimeStamps)
def avglongChangeFreq(listOfLongs):
    if(len(listOfLongs)<1):
        return 0
    return len(set(listOfLongs))/float(len(listOfLongs))
def avgSqrdlongChangeAmt(listOfLongs):
    if(len(listOfLongs)<1):
        return 0
    return np.sum(np.diff(listOfLongs)**2)/float(len(listOfLongs))
def computeEventBasedFeatures(eventsGrpdByDeviceId):
    eventBasedAggregates=eventsGrpdByDeviceId.aggregate({'timestamp':[np.count_nonzero,avgEventDuration,
                            stdEventDuration],'longitude':[avglongChangeFreq,avgSqrdlongChangeAmt]})
    eventBasedAggregatesFeats=['num_of_evnts','avg_evnt_drtn','std_evnt_drtn','avgLongtdChgFrq','avgSqrdLongtdChgAmt']
    eventBasedAggregates.columns=eventBasedAggregatesFeats
    eventBasedAggregates['device_id']=eventBasedAggregates.index
    return (eventBasedAggregates,eventBasedAggregatesFeats) 

In [6]:
(eventBasedFeaturesDF,eventBasedFeats)=computeEventBasedFeatures(eventsGrpdByDeviceId)

In [7]:
def combineDeviceEventsBrandsFeatures(deviceDF,deviceEventsDF,deviceEventsFeats,deviceBrandsDF):
    device_events_brands =deviceDF.merge(deviceEventsDF, how='left',on='device_id')
    imputeDFColsUsingMean(device_events_brands,deviceEventsFeats)
    device_events_brands=device_events_brands.merge(deviceBrandsDF[['phone_brand','device_model','device_id']], 
                                                  how='left',on='device_id')
    return device_events_brands
deviceEvntsBrnds_trainDF=combineDeviceEventsBrandsFeatures(gender_age_train,eventBasedFeaturesDF,
                                                   eventBasedFeats,device_brand_model_trainDF)
deviceEvntsBrnds_testDF=combineDeviceEventsBrandsFeatures(gender_age_test,eventBasedFeaturesDF,
                                                   eventBasedFeats,device_brand_model_testDF)

In [8]:
def processModelFeatures(contFeats,catFeats,modelDF,categorical_features_mask):
    contFeats=list(set(contFeats))
    catFeats=list(set(catFeats))
    modelFeatures=contFeats.copy()
    modelFeatures.extend(catFeats)
    model_subsetDF=modelDF[modelFeatures]
    for contFeat in contFeats:
        scaleFeature(model_subsetDF,contFeat)
    processedModelFeatures=OneHotEncodeFeats(model_subsetDF,catFeats,categorical_features_mask)
    return processedModelFeatures
catFeats=['phone_brand','device_model']
categorical_features_mask=[False,False,False,False,False,True,True]
modelFeats_trainDF=processModelFeatures(eventBasedFeats,catFeats,deviceEvntsBrnds_trainDF,categorical_features_mask)
modelFeats_testDF=processModelFeatures(eventBasedFeats,catFeats,deviceEvntsBrnds_testDF,categorical_features_mask)

/home/alokkumary2j/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/alokkumary2j/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
print(modelFeats_trainDF.shape,modelFeats_testDF.shape)
print(modelFeats_trainDF[11:15,1556:])
deviceEvntsBrnds_trainDF[11:15]

(74645, 1563) (112071, 1658)
[[ 0.          0.          0.42820583  0.22467937  0.10865743  0.09177323
   0.00144613]
 [ 0.          0.          0.23580696  0.37801029  0.02876541  0.08166277
   0.01232859]
 [ 0.          0.          0.09032746  0.18452801  0.          0.04788587
   0.00241022]
 [ 0.          0.          0.23580696  0.37801029  0.02876541  0.08166277
   0.01232859]]


,device_id,gender,age,group,num_of_evnts,avg_evnt_drtn,std_evnt_drtn,avgLongtdChgFrq,avgSqrdLongtdChgAmt,phone_brand,device_model
11,7477216237379271436,F,37,F33-42,7.000000,2.711114e+13,6.637354e+13,0.428571,4084.852943,华为,荣耀6 plus
12,2478205222798310601,F,28,F27-28,52.151315,2.412437e+13,1.116697e+14,0.236296,1081.402878,三星,Galaxy Note 3
13,6352067998666467520,M,32,M32-38,11.000000,1.414618e+13,5.451225e+13,0.090909,0.000000,华为,荣耀畅玩4X
14,-7605360767281960867,M,48,M39+,52.151315,2.412437e+13,1.116697e+14,0.236296,1081.402878,艾优尼,U3


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1,2),min_df=0.0)
device_brand_model_trainDF['brand_model'] = device_brand_model_trainDF['phone_brand'] + ' ' + device_brand_model_trainDF['device_model']
device_brand_model_testDF['brand_model'] = device_brand_model_testDF['phone_brand'] + ' ' + device_brand_model_testDF['device_model']
vect_matrix = vectorizer.fit_transform(device_brand_model_trainDF['brand_model'])
test_vect_matrix = vectorizer.transform(device_brand_model_testDF['brand_model'])

In [11]:
def validateModel(X,y, model):
    kf = KFold(X.shape[0], n_folds=5, shuffle=True, random_state=0)
    for itrain, itest in kf:
        if type(X)==type(pd.DataFrame()):
            train=X.ix[itrain]
            test=X.ix[itest]
        else:
            train = X[itrain,:]
            test = X[itest,:]
        ytrain, ytest = y[itrain], y[itest]
        clf = model.fit(train,ytrain)
        ypred = clf.predict_proba(test)
        print(ypred.shape)
        print(log_loss(ytest, ypred))
        
def getModelOutput(X,y,X2, model):
    kf = KFold(X.shape[0], n_folds=5, shuffle=True, random_state=0)
    for itrain, itest in kf:
        if type(X)==type(pd.DataFrame()):
            train=X.ix[itrain]
            test=X.ix[itest]
        else:
            train = X[itrain,:]
            test = X[itest,:]
        ytrain, ytest = y[itrain], y[itest]
        clf = model.fit(train,ytrain)
        ypred = clf.predict_proba(X2)
        return ypred

In [12]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm  import LinearSVC
#class CalibModel(object):
class CalibModel(object):
    def __init__(self,clf):
        #clf = MultinomialNB()
        print("Obtained Classifier Instance ",clf)
        self.clf = CalibratedClassifierCV(clf, cv=2, method='sigmoid')
    
    def fit(self, X, y):
        self.clf.fit(X,y)
        return self
    
    def predict(self, X):
        return self.clf.predict(X)
    
    def predict_proba(self, X):
        return self.clf.predict_proba(X)

In [13]:
#validateModel(vect_matrix, device_brand_model_trainDF['group'], CalibModel(MultinomialNB()))
#validateModel(vect_matrix, phone_brand_master['group'], CalibModel(LinearSVC()))
#validateModel(eventsData_train_final,eventsData_train.group,CalibModel(GaussianNB()))

In [14]:
#validateModel(modelFeats_trainDF, device_brand_model_trainDF['group'], CalibModel(GaussianNB()))

In [15]:
#from sklearn.ensemble import RandomForestClassifier
#validateModel(modelFeats_trainDF, device_brand_model_trainDF['group'], CalibModel(RandomForestClassifier()))

In [16]:
#validateModel(modelFeats_trainDF, device_brand_model_trainDF['group'], CalibModel(DecisionTreeClassifier()))

In [17]:
#from sklearn.ensemble import AdaBoostClassifier
#validateModel(modelFeats_trainDF, device_brand_model_trainDF['group'], 
#              CalibModel(AdaBoostClassifier(DecisionTreeClassifier())))

In [18]:
modelFeats_trainDF[0]print()

SyntaxError: invalid syntax (<ipython-input-18-f28ec5793bfd>, line 1)